In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from matplotlib import cm
import multiprocessing as mp
from stitch import stitch

In [2]:
# Assume Cobb-Douglas. Generalized later. 

# Household parameters
life = 80
S = 80
working = round(9 * S / 16)
beta_annual = .96
beta = beta_annual ** (life / S)
    
sigma = 3.0

theta_h = 0.5
theta_f = 0.5

nvec_h = np.array([1.0] * working + [0.2] * (S - working))
nvec_f = np.array([1.0] * working + [0.2] * (S - working))

L_h = np.sum(nvec_h)
L_f = np.sum(nvec_f)

# Firm parameters
alpha_h = 0.5
alpha_f = 0.5

Z_h = 1.0
Z_f = 1.0

gamma_h = .35
gamma_f = .35

delta_annual_h = .05
delta_h = 1 - ((1 - delta_annual_h) ** (life / S))

delta_annual_f = .05
delta_f = 1 - ((1 - delta_annual_f) ** (life / S))

# SS parameters
SS_tol = 1e-13
xi = 0.6

In [ ]:
@stitch(lambda cvec, sigma: -sigma / (cvec ** (1 + sigma)), (0, None), 1e-10)
def get_MUvec(cvec, sigma):
    return 1 / (cvec ** sigma)

@stitch(lambda x, gamma: gamma / (x ** (1 - gamma)), (0, None), 1e-10)
def root(x, gamma):
    return x ** gamma

def get_Y(K, L, Z, gamma):
    return Z * (K ** gamma) * (L ** (1 - gamma))

def get_K(r, L, Z, P, gamma, delta):
    return (gamma * Z * P * (L ** (1 - gamma)) / (r + delta)) ** (1 / (1 - gamma))

def get_r(K, L, Z, P, gamma, delta):
    return gamma * Z * P * root(K, (gamma - 1)) * root(L, (1 - gamma)) - delta

def get_w(K, L, Z, P, gamma):
    return (1 - gamma) * Z * P * ((K / L) ** gamma)

def aggregate(theta, P_1, P_2, e):
    constant = ((1 - theta) ** (1 - theta)) * (theta ** theta)
    return (1 / constant) * root(P_1, (1 - theta)) * root((e * P_2), theta)

In [4]:
def get_SS(HH_params, F_params, SS_params, guess_vec):
    S, beta, sigma, theta_h, theta_f, nvec_h, nvec_f, L_h, L_f = HH_params
    Z_h, Z_f, gamma_h, gamma_f, delta_h, delta_f = F_params
    SS_tol, xi = SS_params
    P_h, P_f, r_h, r_f = guess_vec
    
    e = (1 + r_f) / (1 + r_h)
    
    P = aggregate(theta_h, P_h, P_f, 1 / e)
    P_star = aggregate(theta_f, P_f, P_h, e)
    
    r = aggregate(alpha_h, r_h, r_f, 1 / e)
    r_star = aggregate(alpha_f, r_f, r_h, e)
    
    K_h = get_K(r, L_h, Z, P, gamma, delta)
    
    